### XGBoost

In [1]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from keras.optimizers import Adagrad, Adadelta
from time import time
from sklearn.metrics import roc_auc_score
#import xgboost as xgb

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5110)
Using Theano backend.


In [2]:
path = "/input/"
model_path = os.path.join(os.getcwd() + '/models/')
if not os.path.exists(model_path): os.mkdir(model_path)

In [3]:
act_train_data = pd.read_csv("/input/act_train.csv", 
                             dtype={'people_id': np.str, 'activity_id': np.str, 'outcome': np.int8}, 
                             parse_dates=['date'])
act_test_data  = pd.read_csv("/input/act_test.csv", 
                             dtype={'people_id': np.str, 'activity_id': np.str}, parse_dates=['date'])
people_data    = pd.read_csv("/input/people.csv", 
                             dtype={'people_id': np.str, 'activity_id': np.str, 'char_38': np.int32}, 
                             parse_dates=['date'])

In [4]:
act_train_data[:3]

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,outcome
0,ppl_100,act2_1734928,2023-08-26,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 76,0
1,ppl_100,act2_2434093,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
2,ppl_100,act2_3404049,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0


In [5]:
act_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2197291 entries, 0 to 2197290
Data columns (total 15 columns):
people_id            object
activity_id          object
date                 datetime64[ns]
activity_category    object
char_1               object
char_2               object
char_3               object
char_4               object
char_5               object
char_6               object
char_7               object
char_8               object
char_9               object
char_10              object
outcome              int8
dtypes: datetime64[ns](1), int8(1), object(13)
memory usage: 236.8+ MB


In [6]:
people_data[:3]

,people_id,char_1,group_1,char_2,date,char_3,char_4,char_5,char_6,char_7,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,ppl_100,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,...,False,True,True,False,False,True,True,True,False,36
1,ppl_100002,type 2,group 8688,type 3,2021-01-06,type 28,type 9,type 5,type 3,type 11,...,False,True,True,True,True,True,True,True,False,76
2,ppl_100003,type 2,group 33592,type 3,2022-06-10,type 4,type 8,type 5,type 2,type 5,...,False,False,True,True,True,True,False,True,True,99


In [7]:
people_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189118 entries, 0 to 189117
Data columns (total 41 columns):
people_id    189118 non-null object
char_1       189118 non-null object
group_1      189118 non-null object
char_2       189118 non-null object
date         189118 non-null datetime64[ns]
char_3       189118 non-null object
char_4       189118 non-null object
char_5       189118 non-null object
char_6       189118 non-null object
char_7       189118 non-null object
char_8       189118 non-null object
char_9       189118 non-null object
char_10      189118 non-null bool
char_11      189118 non-null bool
char_12      189118 non-null bool
char_13      189118 non-null bool
char_14      189118 non-null bool
char_15      189118 non-null bool
char_16      189118 non-null bool
char_17      189118 non-null bool
char_18      189118 non-null bool
char_19      189118 non-null bool
char_20      189118 non-null bool
char_21      189118 non-null bool
char_22      189118 non-null bool
char_23

In [8]:
print("Train data shape: " + format(act_train_data.shape))
print("Test data shape: " + format(act_test_data.shape))
print("People data shape: " + format(people_data.shape))

Train data shape: (2197291, 15)
Test data shape: (498687, 14)
People data shape: (189118, 41)


In [9]:
def clean_category(dataset, column):
    # summarize those showing only once to one category
    for index, dup in dataset[column].duplicated(keep=False).iteritems():
        if dup == False:
            dataset.set_value(index, column, -1)
    # re-index to continuous numbers
    new_idx = {idx: i for i, idx in enumerate(dataset[column].unique())}
    dataset[column] = dataset[column].apply(lambda x: new_idx[x])
    return dataset

def rescale(dataset, column):
    dataset[column] = dataset[column] - dataset[column].mean() / dataset[column].std()
    return dataset

def act_data_treatment(dataset):
    for col in list(dataset.columns):
        if col not in ['people_id', 'activity_id', 'date', 'char_38', 'outcome']:
            if dataset[col].dtype == 'object':
                # regard NA as a category
                dataset[col].fillna('type 0', inplace=True)
                dataset[col] = dataset[col].apply(lambda x: x.split(' ')[1]).astype(np.int32)
            elif dataset[col].dtype == 'bool':
                # change binary feature to type int (0/1)
                dataset[col] = dataset[col].astype(np.int8)
    
    dataset['year'] = dataset['date'].dt.year
    dataset['month'] = dataset['date'].dt.month
    dataset['day'] = dataset['date'].dt.day
    dataset['isweekend'] = (dataset['date'].dt.weekday >= 5).astype('int8')
    dataset = dataset.drop('date', axis = 1)
    
    return dataset

In [10]:
act_train_data = act_train_data.drop('char_10',axis=1)
act_test_data = act_test_data.drop('char_10',axis=1)

print("Train data shape: " + format(act_train_data.shape))
print("Test data shape: " + format(act_test_data.shape))
print("People data shape: " + format(people_data.shape))

Train data shape: (2197291, 14)
Test data shape: (498687, 13)
People data shape: (189118, 41)


In [11]:
act_train_data = act_data_treatment(act_train_data)
act_test_data  = act_data_treatment(act_test_data)
people_data    = act_data_treatment(people_data)

In [12]:
train = act_train_data.merge(people_data, on='people_id', how='left')
test  = act_test_data.merge(people_data, on='people_id', how='left')
act_id = act_test_data['activity_id']

del act_train_data
del act_test_data
del people_data

In [13]:
print("Train data shape: " + format(train.shape))
print("Test data shape: " + format(test.shape))

Train data shape: (2197291, 60)
Test data shape: (498687, 59)


In [14]:
train = train.drop(['people_id', 'activity_id'], axis = 1)
test = test.drop(['people_id', 'activity_id'], axis = 1)

print("Train data shape: " + format(train.shape))
print("Test data shape: " + format(test.shape))

Train data shape: (2197291, 58)
Test data shape: (498687, 57)


In [15]:
{col:len(train[col].unique()) for col in train.columns}

{'activity_category': 7,
 'char_10': 2,
 'char_11': 2,
 'char_12': 2,
 'char_13': 2,
 'char_14': 2,
 'char_15': 2,
 'char_16': 2,
 'char_17': 2,
 'char_18': 2,
 'char_19': 2,
 'char_1_x': 52,
 'char_1_y': 2,
 'char_20': 2,
 'char_21': 2,
 'char_22': 2,
 'char_23': 2,
 'char_24': 2,
 'char_25': 2,
 'char_26': 2,
 'char_27': 2,
 'char_28': 2,
 'char_29': 2,
 'char_2_x': 33,
 'char_2_y': 3,
 'char_30': 2,
 'char_31': 2,
 'char_32': 2,
 'char_33': 2,
 'char_34': 2,
 'char_35': 2,
 'char_36': 2,
 'char_37': 2,
 'char_38': 101,
 'char_3_x': 12,
 'char_3_y': 43,
 'char_4_x': 8,
 'char_4_y': 25,
 'char_5_x': 8,
 'char_5_y': 9,
 'char_6_x': 6,
 'char_6_y': 7,
 'char_7_x': 9,
 'char_7_y': 25,
 'char_8_x': 19,
 'char_8_y': 8,
 'char_9_x': 20,
 'char_9_y': 9,
 'day_x': 31,
 'day_y': 31,
 'group_1': 29899,
 'isweekend_x': 2,
 'isweekend_y': 2,
 'month_x': 12,
 'month_y': 12,
 'outcome': 2,
 'year_x': 2,
 'year_y': 4}

In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2197291 entries, 0 to 2197290
Data columns (total 58 columns):
activity_category    int32
char_1_x             int32
char_2_x             int32
char_3_x             int32
char_4_x             int32
char_5_x             int32
char_6_x             int32
char_7_x             int32
char_8_x             int32
char_9_x             int32
outcome              int8
year_x               int64
month_x              int64
day_x                int64
isweekend_x          int8
char_1_y             int32
group_1              int32
char_2_y             int32
char_3_y             int32
char_4_y             int32
char_5_y             int32
char_6_y             int32
char_7_y             int32
char_8_y             int32
char_9_y             int32
char_10              int8
char_11              int8
char_12              int8
char_13              int8
char_14              int8
char_15              int8
char_16              int8
char_17              int8
char_1

In [17]:
y = train['outcome']
train = train.drop('outcome',axis=1)

In [18]:
whole = pd.concat([train, test], ignore_index=True)
tr_len = train.shape[0]
del train
del test
categorical = ['group_1', 'activity_category', 'char_1_x', 'char_2_x', 'char_3_x', 'char_4_x', 'char_5_x', 
               'char_6_x', 'char_7_x', 'char_8_x', 'char_9_x', 'char_2_y', 'char_3_y', 'char_4_y', 'char_5_y',
               'char_6_y', 'char_7_y', 'char_8_y', 'char_9_y']
t0 = time()
for category in categorical:
    whole = clean_category(whole, category)
print "Elapsed time %.2f seconds for cleaning categorical features." %(time()-t0)

Elapsed time 75.23 seconds for cleaning categorical features.


In [19]:
# rescale
t0 = time()
to_rescale = ['year_x', 'year_y', 'month_x', 'month_y', 'day_x', 'day_x', 'char_38']
for col in to_rescale:
    whole = rescale(whole, col)
print("Elapsed time %.2f seconds for rescaling." %(time()-t0))

Elapsed time 0.52 seconds for rescaling.


In [20]:
whole['group_1'].unique()[-10:]

array([30920, 30921, 30922, 30923, 30924, 30925, 30926, 30927, 30928, 30929])

In [21]:
# onehot encoding
t0 = time()
whole_cat = []
for category in categorical[1:]:
    whole_cat.append(to_categorical(whole[category]).astype('float32'))
whole_cat = np.concatenate(whole_cat, 1)
print("Elapsed time %.2f seconds for encoding." %(time()-t0))
print("Whole categorical data shape: " + format(whole_cat.shape))

Elapsed time 9.42 seconds for encoding.
Whole categorical data shape: (2695978, 303)


In [22]:
# Split to train, validation, test set of categorical features (excepting group_1)
mask = np.random.rand(tr_len) < 0.2
X_test_cat = whole_cat[tr_len:]
X_cat = whole_cat[:tr_len]
X_val_cat, X_cat = X_cat[mask], X_cat[~mask]
y_val, y = y[mask], y[~mask]

print("Train categorical data shape: " + format(X_cat.shape))
print("Validation categorical data shape: " + format(X_val_cat.shape))
print("Test categorical data shape: " + format(X_test_cat.shape))

del whole_cat

Train categorical data shape: (1757863, 303)
Validation categorical data shape: (439428, 303)
Test categorical data shape: (498687, 303)


In [23]:
# Split to train, validation, test set of 'group_1' feature
whole_group = whole['group_1']
n_groups = whole_group.nunique()
X_test_group = whole_group[tr_len:].astype('int32')
X_group = whole_group[:tr_len].astype('int32')
X_val_group, X_group = X_group[mask], X_group[~mask]

print(len(X_group), len(X_val_group), len(X_test_group))
del whole_group

(1757863, 439428, 498687)


In [24]:
# Split to train, validation, test set of non-categorical features
not_categorical = list(set(whole.columns) - set(categorical))
whole = whole[not_categorical]
X_test_not_cat = whole[tr_len:].values.astype('float32')
X_not_cat = whole[:tr_len].values.astype('float32')
X_val_not_cat, X_not_cat = X_not_cat[mask], X_not_cat[~mask]

print("Train non-categorical data shape: " + format(X_not_cat.shape))
print("Validation non-categorical data shape: " + format(X_val_not_cat.shape))
print("Test non-categorical data shape: " + format(X_test_not_cat.shape))

del whole

Train non-categorical data shape: (1757863, 38)
Validation non-categorical data shape: (439428, 38)
Test non-categorical data shape: (498687, 38)


In [25]:
# combine all data (cat and non-cat) except 'group_1'
X_most = np.concatenate([X_cat, X_not_cat], 1)
X_val_most = np.concatenate([X_val_cat, X_val_not_cat], 1)
X_test_most = np.concatenate([X_test_cat, X_test_not_cat], 1)

print("Train most data shape: " + format(X_most.shape))
print("Validation most data shape: " + format(X_val_most.shape))
print("Test most data shape: " + format(X_test_most.shape))

del X_cat, X_not_cat, X_val_cat, X_val_not_cat, X_test_cat, X_test_not_cat

Train most data shape: (1757863, 341)
Validation most data shape: (439428, 341)
Test most data shape: (498687, 341)


### Training

In [26]:
import keras.callbacks as kcb
class CallMetric(kcb.Callback):
    def on_train_begin(self, logs={}):
        self.best_acc = 0.0
        self.accs = []
        self.val_accs = []
        self.losses = []
        self.val_losses = []
    def on_epoch_end(self, batch, logs={}):
        self.accs.append(logs.get('acc'))
        self.val_accs.append(logs.get('val_acc'))
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        if logs.get('val_acc') > self.best_acc:
            self.best_acc = logs.get('val_acc')
            print("\nThe BEST val_acc to date.")

In [27]:
def build_net(nb_neurons, embedding_dim):
    most_in = Input(shape=(X_most.shape[1],), dtype='float32', name='most_in')
    group_in = Input(shape=(1,), dtype='int32', name='group_in')
    group_emb = Embedding(n_groups, embedding_dim, input_length=1, W_regularizer=l2(1e-4))(group_in)
    group_emb = Reshape((embedding_dim,))(group_emb)
    group_emb = BatchNormalization()(group_emb)
    most_feature = BatchNormalization()(most_in)
    x = merge([most_feature, group_emb], mode='concat')
    for n in nb_neurons:
        x = Dense(n, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.2)(x)
    x = Dense(1, activation='sigmoid')(x)
    net = Model([most_in, group_in], x)
    net.compile(Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return net

In [28]:
metricRecords = CallMetric()
checkpointer = kcb.ModelCheckpoint(filepath=model_path+"net_redhat.h5", monitor='val_acc', verbose=1, save_best_only=True)
net = build_net([256], 128)

In [61]:
net.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
group_in (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_5 (Embedding)          (None, 1, 128)        3959040     group_in[0][0]                   
____________________________________________________________________________________________________
most_in (InputLayer)             (None, 341)           0                                            
____________________________________________________________________________________________________
reshape_5 (Reshape)              (None, 128)           0           embedding_5[0][0]                
___________________________________________________________________________________________

In [62]:
net.fit([X_most, X_group], y, batch_size=512, nb_epoch=8, 
          validation_data=([X_val_most, X_val_group], y_val), callbacks=[metricRecords, checkpointer])

Train on 1757953 samples, validate on 439338 samples
Epoch 1/8
1756672/1757953 [============================>.] - ETA: 0s - loss: 0.1080 - acc: 0.9709  

In [63]:
net.load_weights(model_path+'net_redhat.h5')
K.set_value(net.optimizer.lr, 1e-4)
net.fit([X_most, X_group], y, batch_size=512, nb_epoch=3, 
          validation_data=([X_val_most, X_val_group], y_val), callbacks=[metricRecords, checkpointer])

Train on 1757953 samples, validate on 439338 samples
Epoch 1/3
1756672/1757953 [============================>.] - ETA: 0s - loss: 0.0192 - acc: 0.9964  

In [64]:
net.load_weights(model_path+'net_redhat.h5')
K.set_value(net.optimizer.lr, 1e-5)
net.fit([X_most, X_group], y, batch_size=512, nb_epoch=3, 
          validation_data=([X_val_most, X_val_group], y_val), callbacks=[metricRecords, checkpointer])

Train on 1757953 samples, validate on 439338 samples
Epoch 1/3
1757184/1757953 [============================>.] - ETA: 0s - loss: 0.0106 - acc: 0.9978  

In [66]:
randSample = choice(len(X_most), 200000, replace=False)
y_tr = y.values[randSample]
net.load_weights(model_path+'net_redhat.h5')
pred_tr = net.predict([X_most[randSample], X_group.values[randSample]], batch_size=512)
print('Train accuracy:', np.mean(y_tr[:,None] == (pred_tr>=0.5).astype('int8')))
print('Train auc:', roc_auc_score(y_tr, pred_tr))

In [52]:
pred_val = net.predict([X_val_most, X_val_group], batch_size=512)
print('Valid accuracy:', np.mean(y_val[:,None] == (pred_val>=0.5).astype('int8')))
print('Valid auc:', roc_auc_score(y_val, pred_val))

In [66]:
roc_auc_score(y_val, pred_val)

0.9997939448013462

In [67]:
np.mean(y_val[:,None] == (pred_val>=0.5).astype('int8'))

0.99615330337917507

In [74]:
pred = net.predict([X_test_most, X_test_group], batch_size=512)

In [75]:
from IPython.lib.display import FileLink
from IPython.display import HTML

output = pd.DataFrame({ 'activity_id': act_id, 'outcome': pred.ravel() })
output.head()
output.to_csv('submit_redhat.csv', index=False)
#FileLink('submit_redhat.csv')
HTML('<p>The submission is <a href="submit_redhat.csv">here</a>.</p>')

In [124]:
def build_net(nb_neurons, embedding_dim):
    most_in = Input(shape=(X_most.shape[1],), dtype='float32', name='most_in')
    group_in = Input(shape=(1,), dtype='int32', name='group_in')
    group_emb = Embedding(n_groups, embedding_dim, input_length=1)(group_in)
    group_emb = Reshape((embedding_dim,))(group_emb)
    group_emb = BatchNormalization()(group_emb)
    most_feature = BatchNormalization()(most_in)
    x = merge([most_feature, group_emb], mode='concat')
    for n in nb_neurons:
        x = Dense(n, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.25)(x)
    x = Dense(1, activation='sigmoid')(x)
    net = Model([most_in, group_in], x)
    net.compile(Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return net

In [127]:
metricRecords = CallMetric()
checkpointer = kcb.ModelCheckpoint(filepath=model_path+"net2_redhat.h5", monitor='val_acc', verbose=1, save_best_only=True)
net = build_net([256], 128)

In [115]:
net.fit([X_most, X_group], y, batch_size=512, nb_epoch=4, 
          validation_data=([X_val_most, X_val_group], y_val), callbacks=[metricRecords, checkpointer])

Train on 1757863 samples, validate on 439428 samples
Epoch 1/4
1757696/1757863 [============================>.] - ETA: 0s - loss: 0.0782 - acc: 0.9681  

In [128]:
net.load_weights(model_path+"net2_redhat.h5")
K.set_value(net.optimizer.lr, 1e-4)
net.fit([X_most, X_group], y, batch_size=512, nb_epoch=2, 
          validation_data=([X_val_most, X_val_group], y_val), callbacks=[metricRecords, checkpointer])

Train on 1757863 samples, validate on 439428 samples
Epoch 1/2
1756160/1757863 [============================>.] - ETA: 0s - loss: 0.0078 - acc: 0.9970  

In [130]:
net.load_weights(model_path+'net2_redhat.h5')
pred_tr = net.predict([X_most[randSample], X_group.values[randSample]], batch_size=512)
print('Train accuracy:', np.mean(y_tr[:,None] == (pred_tr>=0.5).astype('int8')))
print('Train auc:', roc_auc_score(y_tr, pred_tr))

('Train accuracy:', 0.99849500000000002)
('Train auc:', 0.99999071826495745)


In [131]:
pred_val = net.predict([X_val_most, X_val_group], batch_size=512)
print('Valid accuracy:', np.mean(y_val[:,None] == (pred_val>=0.5).astype('int8')))
print('Valid auc:', roc_auc_score(y_val, pred_val))

('Valid accuracy:', 0.99646358447800321)
('Valid auc:', 0.99966190493243523)


In [113]:
def build_net(nb_neurons, embedding_dim):
    most_in = Input(shape=(X_most.shape[1],), dtype='float32', name='most_in')
    group_in = Input(shape=(1,), dtype='int32', name='group_in')
    group_emb = Embedding(n_groups, embedding_dim, input_length=1)(group_in)
    group_emb = Reshape((embedding_dim,))(group_emb)
    group_emb = BatchNormalization()(group_emb)
    most_feature = BatchNormalization()(most_in)
    x = merge([most_feature, group_emb], mode='concat')
    for n in nb_neurons:
        x = Dense(n, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
    x = Dense(1, activation='sigmoid')(x)
    net = Model([most_in, group_in], x)
    net.compile(Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return net

In [114]:
metricRecords = CallMetric()
checkpointer = kcb.ModelCheckpoint(filepath=model_path+"net4_redhat.h5", monitor='val_acc', verbose=1, save_best_only=True)
net = build_net([256], 256)

In [115]:
net.fit([X_most, X_group], y, batch_size=512, nb_epoch=4, 
          validation_data=([X_val_most, X_val_group], y_val), callbacks=[metricRecords, checkpointer])

Train on 1757863 samples, validate on 439428 samples
Epoch 1/4
1757696/1757863 [============================>.] - ETA: 0s - loss: 0.0782 - acc: 0.9681  

In [119]:
net.load_weights(model_path+"net4_redhat.h5")
K.set_value(net.optimizer.lr, 1e-4)
net.fit([X_most, X_group], y, batch_size=512, nb_epoch=2, 
          validation_data=([X_val_most, X_val_group], y_val), callbacks=[metricRecords, checkpointer])

Train on 1757863 samples, validate on 439428 samples
Epoch 1/2
1756672/1757863 [============================>.] - ETA: 0s - loss: 0.0080 - acc: 0.9969  

In [120]:
net.load_weights(model_path+'net4_redhat.h5')
pred_tr = net.predict([X_most[randSample], X_group.values[randSample]], batch_size=512)
print('Train accuracy:', np.mean(y_tr[:,None] == (pred_tr>=0.5).astype('int8')))
print('Train auc:', roc_auc_score(y_tr, pred_tr))

('Train accuracy:', 0.99855499999999997)
('Train auc:', 0.99999061310039583)


In [121]:
pred_val = net.predict([X_val_most, X_val_group], batch_size=512)
print('Valid accuracy:', np.mean(y_val[:,None] == (pred_val>=0.5).astype('int8')))
print('Valid auc:', roc_auc_score(y_val, pred_val))

('Valid accuracy:', 0.9966069526748409)
('Valid auc:', 0.99963187718180202)


In [132]:
pred = net.predict([X_test_most, X_test_group], batch_size=512)

In [133]:
from IPython.lib.display import FileLink
from IPython.display import HTML

output = pd.DataFrame({ 'activity_id': act_id, 'outcome': pred.ravel() })
output.head()
output.to_csv('submit_redhat.csv', index=False)
#FileLink('submit_redhat.csv')
HTML('<p>The submission is <a href="submit_redhat.csv">here</a>.</p>')

### Ensembling

In [27]:
def build_net(nb_layers, nb_neurons, embedding_dim):
    most_in = Input(shape=(X_most.shape[1],), dtype='float32', name='most_in')
    group_in = Input(shape=(1,), dtype='int32', name='group_in')
    group_emb = Embedding(n_groups, embedding_dim, input_length=1, W_regularizer=l2(1e-4))(group_in)
    group_emb = Reshape((embedding_dim,))(group_emb)
    group_emb = BatchNormalization()(group_emb)
    most_feature = BatchNormalization()(most_in)
    x = merge([most_feature, group_emb], mode='concat')
    for _ in range(nb_layers):
        x = Dense(nb_neurons, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.2)(x)
    x = Dense(1, activation='sigmoid')(x)
    net = Model([most_in, group_in], x)
    net.compile(Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return net

In [28]:
metricRecords = CallMetric()
checkpointer = kcb.ModelCheckpoint(filepath=model_path+"net_redhat.h5", monitor='val_acc', verbose=1, save_best_only=True)
net = build_net(1, 256, 128)

In [ ]:
def fit_model(index, nb_layer, nb_neuron, embed_dim):
    path = model_path+"net"+str(index)+"_redhat.h5"
    net = build(nb_layer, nb_neuron, embed_dim)
    checkpointer = kcb.ModelCheckpoint(filepath=path, monitor='val_acc', verbose=1, save_best_only=True)
    net.fit([X_most, X_group], y, batch_size=512, nb_epoch=8, 
          validation_data=([X_val_most, X_val_group], y_val), callbacks=[checkpointer])
    K.set_value(net.optimizer.lr, 1e-4)
    net.fit([X_most, X_group], y, batch_size=512, nb_epoch=2, 
          validation_data=([X_val_most, X_val_group], y_val), callbacks=[checkpointer])
    net.load_weights(path)
    return net

In [ ]:
models = [fit_model(1, 256, 256), fit_model(2, 128, 256), fit_model(3, 64, 256)]

In [ ]:
preds = np.stack([model.predict([X_test_most, X_test_group], batch_size=512) for model in models])

In [ ]:
avg_pred = np.mean(preds, axis=0)

In [75]:
from IPython.lib.display import FileLink
from IPython.display import HTML

output = pd.DataFrame({ 'activity_id': act_id, 'outcome': avg_pred.ravel() })
output.head()
output.to_csv('submit_redhat.csv', index=False)
#FileLink('submit_redhat.csv')
HTML('<p>The submission is <a href="submit_redhat.csv">here</a>.</p>')

### Pseudo-labling

In [76]:
net = build_net()
net.load_weights(model_path+'net_redhat.h5')
K.set_value(net.optimizer.lr, 1e-3)
net.fit([np.concatenate([X_most[randSample], X_test_most[:10000]]), 
         np.concatenate([X_group.values[randSample], X_test_group.values[:10000]])], 
        np.concatenate([y_tr, pred.ravel()[:10000]]), batch_size=512, nb_epoch=3, 
          validation_data=([X_val_most, X_val_group], y_val))

Train on 210000 samples, validate on 440919 samples
Epoch 1/3
210000/210000 [==============================] - 13s - loss: 0.0942 - acc: 0.9198 - val_loss: 0.0601 - val_acc: 0.9787

In [37]:
y.values.shape, pred.shape

((1756372,), (498687, 1))

In [55]:
def pseudo_labeled_set(train_set, test_set, batch_size=512, ratio=0.8, arrange=True):
    X_tr_most, X_tr_group, y_tr = train_set
    
    true_part = int(batch_size*ratio)
    pseudo_part = batch_size - true_part
    tr_len = len(y_tr)
    nb_batch = tr_len // true_part
    ps_len = nb_batch * pseudo_part
    
    mask = choice(range(len(test_set[0])), ps_len, replace=False)
    X_ps_most, X_ps_group, y_ps = test_set[0][mask], test_set[1][mask], test_set[2][mask]
    
    if arrange:
        X_most_new = np.concatenate(
            [np.concatenate([X_tr_most[i*true_part: i*true_part+true_part], 
                             X_ps_most[i*pseudo_part: i*pseudo_part+pseudo_part]]) for i in range(nb_batch)])
        X_group_new = np.concatenate(
            [np.concatenate([X_tr_group[i*true_part: i*true_part+true_part], 
                             X_ps_group[i*pseudo_part: i*pseudo_part+pseudo_part]]) for i in range(nb_batch)])
        y_new = np.concatenate(
            [np.concatenate([y_tr[i*true_part: i*true_part+true_part], 
                             y_ps[i*pseudo_part: i*pseudo_part+pseudo_part]]) for i in range(nb_batch)])
        
        return X_most_new, np.array(X_group_new), y_new
    else:
        rand_arrange = np.random.permutation(tr_len + ps_len)
        return np.concatenate([X_tr_most, X_ps_most])[rand_arrange], \
                np.concatenate([X_tr_group, X_ps_group])[rand_arrange], \
                np.concatenate([y_tr, y_ps])[rand_arrange]

In [56]:
X_ps_most, X_ps_group, y_ps = pseudo_labeled_set((X_most, X_group.values, y.values), 
                                            (X_test_most, X_test_group.values, (pred.ravel()>=0.5).astype('int32')), 
                                                 arrange=True)

In [50]:
X_ps_most.shape, X_ps_group.shape, y_ps.shape

((2195456, 341), (2195456,), (2195456,))

In [57]:
X_ps_most.shape, X_ps_group.shape, y_ps.shape

((2198528, 341), (2198528,), (2198528,))

In [58]:
net.load_weights(model_path+'net_redhat.h5')
metricRecords = CallMetric()
checkpointer = kcb.ModelCheckpoint(filepath=model_path+"net_pseudo.h5", monitor='val_acc', verbose=1, save_best_only=True)
net = build_net()
K.set_value(net.optimizer.lr, 1e-4)
net.fit([X_ps_most, X_ps_group], y_ps, batch_size=512, nb_epoch=1, 
          validation_data=([X_val_most, X_val_group], y_val), callbacks=[metricRecords, checkpointer])

Train on 2198528 samples, validate on 440919 samples
Epoch 1/1
2198016/2198528 [============================>.] - ETA: 0s - loss: 0.1850 - acc: 0.9539  